# MAX-CUT inference

In [ ]:
pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install cvxpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.linalg import sqrtm
import cvxpy as cp
from sklearn import preprocessing as pp
import time

## Generate a random graph and solve the Linear Programming Problem

In [ ]:
n = int(input("Enter number of nodes in graph : "))
p = float(input("Enter the probability of edge between nodes : "))
seed = int(input("Enter seed value to generate random graph : "))

Enter number of nodes in graph : 200
Enter the probability of edge between nodes : 0.5
Enter seed value to generate random graph : 234


In [ ]:
graph = nx.gnp_random_graph(n, p, seed = seed)

In [ ]:
# print("Graph is : ")
# nx.draw(graph)

In [ ]:
adj =nx.to_numpy_array(graph)
adj

array([[0., 1., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 1.],
       [1., 0., 1., ..., 0., 1., 0.]])

## Converting this to a Binary Quadratic Optimization Problem

In [ ]:
class BinaryQP(object) : 
    def __init__(self, graph) : 
        
        self.graph = graph
        
        self.adj = nx.to_numpy_array(graph)
        self.n = len(adj)
        
        self.pot = np.array([0 for i in range(self.n)])
        self.constant = 0
        
        cost_matrix = np.zeros((n+1, n+1))
        cost_matrix[np.ix_(list(range(n)),list(range(n)))] = self.adj
        cost_matrix[np.ix_([n],list(range(n)))] = [i/2 for i in self.pot]
        cost_matrix[np.ix_(list(range(n))),[n]] = [i/2 for i in self.pot]
        cost_matrix[n][n] = self.constant
        
        self.cost_matrix = cost_matrix
        
    def __str__(self) : 
        print("The Cost Matrix (S) is :")
        return str(self.cost_matrix)

In [ ]:
problem = BinaryQP(graph)
print(problem)

The Cost Matrix (S) is :
[[0. 1. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 1. 0. ... 0. 1. 0.]
 [1. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Solving using Semidefinite programming

In [ ]:
def solveUsingSDP(problem : BinaryQP) : 
    cost_matrix = problem.cost_matrix
    n = len(cost_matrix)
    X = cp.Variable((n, n), symmetric = True)
    obj = cp.trace(cost_matrix*(1-X))
    constraints = [
        cp.diag(X) == 1
    ]
    constraints += [
        X - 0.0001*np.eye(n) >> 0
    ]
    prob = cp.Problem(cp.Maximize(obj), constraints)
    print('started SDp')
    prob.solve(solver='SCS')
    L = np.linalg.cholesky(X.value)
    R = np.random.normal(size=(n,1))
    rounded = np.dot(L, R)
    labels = np.sign(rounded[:,0])
    # print("Optimal value: ", prob.value)
    return labels, prob.value

In [ ]:
labels, optValue = solveUsingSDP(problem)

/usr/local/lib/python3.8/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, UserWarning)


started SDp


In [ ]:
print(optValue)

22524.943119943804


In [ ]:
print(labels)

[ 1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
  1. -1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.
  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1. -1.
 -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.
 -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1.  1.  1.
 -1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1.
  1. -1.  1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1.
  1.  1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1. -1.
  1.  1.  1.  1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1.
 -1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1.
  1. -1. -1.]


#### Finding the Cut value for the solved problm

In [ ]:
res = 0
for n1 in range(n) : 
  for n2 in range(n1 + 1, n) : 
    if adj[n1][n2]  and labels[n1] != labels[n2] : 
      res += 1
print(res)

5457


In [ ]:
print(res)

5457


In [ ]:
print(len(labels))

201


## Solving using EP-SDP

In [ ]:
def solveUsingEPSDP(problem, rank,la, ga, steps):
    C = problem.cost_matrix
    D = np.diag(np.sum(C, axis=0))
    # print(D)
    L = D - C
    C = L
    V = np.random.normal(0, 1, (rank, len(C)))
    V = np.transpose(pp.normalize(np.transpose(V), norm='l2'))
    step = 1/np.linalg.norm(C) 
    
    
    for _ in range(10) : 
        # print('outer')
        gradient = 2*np.matmul(V, C) - la*PenGrad(V) 
        print(np.trace(gradient))
        # for _ in range(steps):
            # print('inner')
            gradient = 2*np.matmul(V, C) - la*PenGrad(V); 
            V = V + step*(gradient)
            V = np.transpose(pp.normalize(np.transpose(V), norm='l2'))
        
        la = la*ga
        
    U, s, Vt = np.linalg.svd(V)
    v1 = Vt.T[:, 0]
    return v1



def PenGrad(V):
    alpha = 3
    U1, d, U2 = np.linalg.svd(V, full_matrices=False)
    D = np.diag(d)    
    return (alpha/(1-alpha))*(np.dot(U1, np.dot(np.linalg.matrix_power(D, 2*alpha-1),U2))/np.power(np.trace(np.dot(D, D)), alpha) -  np.trace(np.linalg.matrix_power(D, 2*alpha))*V/np.power(np.trace(np.dot(D, D)), alpha+1) )
    

In [ ]:
res = solveUsingEPSDP(problem, 20, 10, 5, 100 )

27.817861565536464
27.81810133668977
27.819300192456424
27.825294471289673
27.855265865455742


In [ ]:
res.shape

(201,)

In [ ]:
print(res)

[-0.05306015 -0.01299321  0.02579894  0.05674399  0.01268715  0.0444277
  0.03078239  0.04316983 -0.00065947  0.07217825 -0.10921415 -0.14810116
  0.01734227  0.05846288 -0.06529645 -0.04157142  0.0033221   0.07021152
  0.08937798 -0.06399805 -0.00779027 -0.04044416 -0.00377362  0.07626101
  0.07503891  0.14784361 -0.04609257  0.10417016 -0.0628929   0.01697519
  0.07049672  0.12602157  0.08199108 -0.06322781 -0.00451798 -0.05774656
 -0.10401607  0.03772999  0.09630433  0.13963948 -0.06422262 -0.08559441
 -0.08796865 -0.0296858   0.04620658 -0.00822003 -0.06807049 -0.01460727
 -0.11631298  0.02366842 -0.04558723  0.0010766   0.05434226  0.1161658
 -0.0793763  -0.02129124 -0.03501578 -0.08969728  0.00292327  0.04358177
 -0.07533465 -0.07904992 -0.01945048  0.00771457 -0.01782239  0.12392001
  0.03015386 -0.02894396  0.04572297 -0.00757725 -0.03515921 -0.07399519
  0.09969879 -0.04741964 -0.03980034 -0.07395476  0.1009776   0.01641305
  0.05043124  0.06143724  0.09772909 -0.04623302  0.0

In [ ]:

r = np.random.randn(n + 1)
# print(r)
print(r.shape, res.shape)
print(r * res)
lables = np.sign(r * res)
print(lables)

(201,) (201,)
[ 2.09901031e-02 -3.17329174e-03 -5.48464418e-03  4.26572109e-02
  1.10625188e-02 -1.42542265e-02  2.63481803e-02 -4.51317916e-02
 -7.10725281e-04 -8.85154737e-02  1.16305274e-01 -2.68617623e-01
  3.58545703e-03 -9.50290247e-02  2.15419416e-02 -6.90043319e-02
  5.00188006e-03  2.35028741e-02 -1.91713836e-01 -1.20935419e-01
  5.24822480e-03 -1.98881968e-02  8.87657971e-03  2.73599854e-02
  8.07707692e-03 -7.62109703e-02  6.76503009e-02 -1.71130872e-02
 -3.08076069e-02 -4.14914479e-03 -7.57478399e-02 -1.05064558e-03
 -9.42536318e-02 -1.82833233e-02 -3.28180040e-03  3.53881127e-02
  1.60889686e-01  2.64047376e-02  1.22749695e-02 -1.47575288e-02
 -1.38616203e-03 -4.31098445e-02 -1.21070939e-04  1.52565575e-02
 -2.83535386e-02 -5.03910667e-03  6.14308329e-02  2.24736893e-02
  5.33860486e-02  3.92867333e-02  1.42120222e-02  3.08557645e-03
  7.67983625e-02 -7.06890288e-03 -3.80680959e-02 -4.12262800e-02
 -2.55253361e-02 -4.56394286e-02 -2.59928530e-03 -7.72197960e-02
 -4.2084333

In [ ]:
lables = list(lables)
res = 0
for n1 in range(n) : 
  for n2 in range(n1 , n) : 
    if adj[n1][n2]  and lables[n1] != lables[n2] : 
      res += 1
print(res)

4970
